# Project Title: Property Usage
## By Go Code CO team "Tech Slope" - Aaron, Adam, Dex, Leo, Marc
### Version 1.1
### Last update: May 20, 2019

In [1]:
### Initiate
import pandas as pd
import geopandas as gpd
import time
import numpy as np
import matplotlib as plt
from matplotlib.path import Path
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder,LabelBinarizer

from google.colab import drive
drive.mount('/content/gdrive')

     |████████████████████████████████| 901kB 1.7MB/s 
     |████████████████████████████████| 17.9MB 1.5MB/s 
     |████████████████████████████████| 10.8MB 40.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built munch


Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## Task 1 Training with Denver Data
Main parameters: Area sqft, zoning code <br>
Result: Property subtype provided by City of Denver <br>

*Note: Although we have processed crime, population, and traffic data, it seems that the only golden features are just the 'zoning code' and 'square footage' to determine the property usage type. We might put these attributes back to the model if we have a series of well-defined subtypes to use.*

In [0]:
## Load data
workdir='gdrive/My Drive/GoCode - PropertyUsage/Data processing/'
filename=workdir+'Clean Data/DenverParcels_ML.csv'
df = pd.read_csv(filename,na_values='None',dtype={"Area":np.float64,'PropertySubtype':'category'})
df=df.dropna(axis=0,how='any')
subtypelist = df['PropertySubtype'].unique()
num_classes = subtypelist.shape[0]

## Encode and normalize inputs to 0-1
lb = LabelBinarizer()
df_Y = lb.fit_transform(df['PropertySubtype'])
lb_make = LabelEncoder()
df["Zoning_Code"] = lb_make.fit_transform(df["ZoningCode"])
df_X = df.drop(columns=['PropertySubtype','ZoningCode'])

df_X_max =df_X['Area'].max()
df_X_min =df_X['Area'].min()
df_X['AreaNormalized']=(df_X['Area']-df_X_min)/(df_X_max-df_X_min)
df_X.drop(columns='Area',inplace=True)
df_X.head()

,Zoning_Code,AreaNormalized
0,168,0.000168
1,166,0.000179
2,168,0.000112
3,75,0.000110
4,168,0.000238


In [0]:
df.head()

,PropertySubtype,ZoningCode,Area,Zoning_Code
0,SINGLE FAMILY,U-SU-B1,4690.0,168
1,SINGLE FAMILY,U-SU-A1,5000.0,166
2,SINGLE FAMILY,U-SU-B1,3140.0,168
3,SINGLE FAMILY,G-MU-3,3060.0,75
4,SINGLE FAMILY,U-SU-B1,6650.0,168


In [5]:
# Create a model
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical

## Fix random seed for reproducibility
seed = 42
np.random.seed(seed)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test= train_test_split(df_X, df_Y, test_size=0.20, random_state=seed)
X_cols=X_train.shape[1]


def MultilabelClassification_model():
### YOUR TURN
    # Create a model with 2 convolutional layers and 1 dense layers before the output
    # Use Batch Normalization and Adam optimizer
    model = Sequential()
    model.add(Dense(155, activation='relu', input_shape=(X_cols,)))
    model.add(Dense(155, activation='relu'))
    model.add(Dense(num_classes))
    model.add(BatchNormalization())
    ####model.add(Activation('softmax')) # no good result 
    ####model.compile(loss= 'categorical_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
    model.add(Activation('sigmoid'))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
    return model
# build the model
model=MultilabelClassification_model()

NameError: ignored

In [3]:
# Fit the model
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epoch=20, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, Y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

NameError: ignored

In [0]:
code=170
area=0.000102
test=np.array([pd.Series([code, area]),])
test.shape
model.predict(test)


array([[3.5682321e-04, 3.9857626e-04, 1.7911196e-04, 6.9817901e-04,
        2.1919608e-04, 4.4052601e-03, 2.0107627e-04, 3.0243397e-04,
        2.2195131e-02, 3.2537878e-03, 5.9964359e-03, 9.1224909e-04,
        1.0428131e-03, 3.3804774e-04, 3.8772821e-04, 2.5290251e-04,
        2.1451712e-04, 1.6742945e-04, 2.3162365e-04, 1.9744039e-04,
        1.5676022e-04, 2.1606684e-04, 1.9278228e-03, 2.0158291e-04,
        1.9618869e-04, 1.6623735e-04, 1.6033649e-04, 1.4930964e-04,
        1.7538667e-04, 2.1409988e-04, 1.9314885e-04, 2.7248263e-04,
        1.9982457e-04, 2.4434328e-03, 2.6020408e-04, 1.6334653e-04,
        1.5074015e-04, 3.0090004e-02, 3.6770105e-04, 9.6485019e-04,
        3.1819940e-04, 2.7480721e-04, 2.0578504e-04, 7.1078539e-04,
        1.7923117e-04, 1.5592575e-04, 4.0808320e-04, 1.5157461e-04,
        1.5017390e-04, 2.4470687e-04, 2.5853515e-04, 4.1785836e-04,
        3.3968687e-04, 3.0514598e-04, 1.5947223e-04, 1.5726686e-04,
        1.6921759e-04, 2.8714538e-04, 2.0596385e

## Task 3 Making Predictions with Denver Data

In [2]:
X_train, Y_train, validation_data=(X_test, Y_test)

NameError: ignored

## Task 2 Find zoning for each Mesa County address


In [0]:
## Match Opportunity Zones
df_mesazone = pd.read_csv('Address_Mesa_Cleaned.csv',dtype={'StreetNumber':str}) ## 49849 rows
df_mesazone['OppZoneGeoID']='0'
df_mesazone['EntZone']=False
df_mesazone['ZoningCode']='0'
df_mesazone.head()

,Latitude,Longitude,StreetNumber,StreetName,Unit,City,State,ZIP,FullAddress,OppZoneGeoID,EntZone,ZoningCode
0,38.523159,-108.889763,56500,HWY 141,NaN,GATEWAY,CO,81522,56500 HWY 141,0,False,0
1,38.534833,-108.896495,55002,HWY 141,NaN,GATEWAY,CO,81522,55002 HWY 141,0,False,0
2,38.535193,-108.838723,34165,S 12 8/10 RD,NaN,WHITEWATER,CO,81527,34165 S 12 8/10 RD,0,False,0
3,38.541657,-108.567979,30200,UNCOMPAHGRE DIVIDE RD,NaN,WHITEWATER,CO,815XX,30200 UNCOMPAHGRE DIVIDE RD,0,False,0
4,38.563368,-108.921568,52248,HWY 141,NaN,GATEWAY,CO,81522,52248 HWY 141,0,False,0


In [0]:
for i in range(len(df_mesazone)):
    point=Point(df_mesazone['Longitude'][i],df_mesazone['Latitude'][i])
    for j in range(len(df_opp)): 
        if df_opp['geometry'][j].contains(point):
            df_mesazone['OppZoneGeoID'][i]= df_opp['GEOID'][j]
            break
##    if (i%1000==0)and(df_mesazone['OppZoneGeoID'][i]!='0'): print(i,df_mesazone['OppZoneGeoID'][i])      


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


13000 08077000800
14000 08077000800
15000 08077000900
17000 08077000200
19000 08077000800
21000 08077000800
25000 08077000200
28000 08077000800
29000 08077000700
30000 08077000500
31000 08077000800
34000 08077000500
37000 08077000400
41000 08077000400
47000 08077000400
48000 08077000400


,Latitude,Longitude,StreetNumber,StreetName,Unit,City,State,ZIP,FullAddress,OppZoneGeoID,EntZone,ZoningCode
49844,39.089047,-108.553640,1156.0,BOOKCLIFF AVE,12,GRAND JUNCTION,CO,81501,1156.0 BOOKCLIFF AVE,08077000500,False,N/A
49845,39.087296,-108.471576,550.0,WARRIOR WAY,302,GRAND JUNCTION,CO,81504,550.0 WARRIOR WAY,00000000000,False,N/A
49846,39.088367,-108.521428,2864.0,PRESLEY AVE,NaN,GRAND JUNCTION,CO,81501,2864.0 PRESLEY AVE,00000000000,False,N/A
49847,39.087289,-108.471221,550.0,WARRIOR WAY,312,GRAND JUNCTION,CO,81504,550.0 WARRIOR WAY,00000000000,False,N/A
49848,NaN,-108.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00000000000,False,N/A


In [0]:
### Match enterprise zones:
for i in range(len(df_mesazone)):
    point=Point(df_mesazone['Longitude'][i],df_mesazone['Latitude'][i])
    for j in range(len(df_ent)): 
        if df_ent['geometry'][j].contains(point):
            df_mesazone['EntZone'][i]= True
            break
    if (i%1000==0)and(df_mesazone['EntZone'][i]==True): print(i,df_mesazone['EntZone'][i])  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


2000 True
3000 True
4000 True
5000 True
6000 True
7000 True
8000 True
9000 True
10000 True
12000 True
13000 True
14000 True
15000 True
16000 True
17000 True
18000 True
19000 True
20000 True
21000 True
22000 True
23000 True
24000 True
25000 True
26000 True
27000 True
28000 True
29000 True
30000 True
31000 True
32000 True
33000 True
34000 True
35000 True
36000 True
37000 True
38000 True
39000 True
41000 True
42000 True
44000 True
45000 True
46000 True
47000 True
48000 True
49000 True


In [0]:
def write_checkpoint(df, dfname):
  filename=time.strftime("%Y%m%d-%H%M")+' '+ dfname+'.csv'
  df.to_csv(filename, index=False)
  return

def read_checkpoint(filename):
    return pd.read_csv(filename,index_col=0)
  
write_checkpoint(df_mesazone,'Mesa_Zone')
## df_mesazone = pd.read_csv('Mesa_Zone.csv',dtype={"StreetNumber":str,'OppZoneGeoID':str})
## df_mesazone.head()

,Latitude,Longitude,StreetNumber,StreetName,Unit,City,State,ZIP,FullAddress,OppZoneGeoID,EntZone,ZoningCode
0,38.523159,-108.889763,56500,HWY 141,NaN,GATEWAY,CO,81522,56500 HWY 141,00000000000,False,0
1,38.534833,-108.896495,55002,HWY 141,NaN,GATEWAY,CO,81522,55002 HWY 141,00000000000,False,0
2,38.535193,-108.838723,34165,S 12 8/10 RD,NaN,WHITEWATER,CO,81527,34165 S 12 8/10 RD,00000000000,False,0
3,38.541657,-108.567979,30200,UNCOMPAHGRE DIVIDE RD,NaN,WHITEWATER,CO,815XX,30200 UNCOMPAHGRE DIVIDE RD,00000000000,False,0
4,38.563368,-108.921568,52248,HWY 141,NaN,GATEWAY,CO,81522,52248 HWY 141,00000000000,False,0


In [0]:
### Match zoning:
for i in range(len(df_mesazone)):
    point=Point(df_mesazone['Longitude'][i],df_mesazone['Latitude'][i])
    for j in range(len(df_zoning_gj)): 
        if df_zoning_gj['geometry'][j].contains(point):
            df_mesazone['ZoningCode'][i]= df_zoning_gj['Name'][j]
            if i%1000==0: print(i,j,df_zoning_gj['Name'][j])
            break  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


0 326 AFT
1000 326 AFT
2000 479 PUD
3000 785 RSF-4
4000 995 RSF-4


TopologyException: side location conflict at -108.529228 39.044763000000003


TopologicalError: ignored

In [0]:
''' Point Check
point=Point(-108.56177,39.0676422)
for j in range(len(df_zoning_gj)):  ##1005 records, 10s per point
    if df_zoning_gj['geometry'][j].contains(point):
        print(df_zoning_gj['Name'][j])
df_mesazone.iloc[1109]
'''

In [0]:
print(df_mesazone.tail(),'\n','---------','\n')
print('Shape of the current dataframe:',df_mesazone.shape,'\n')
print('Missing value counts:')
print(df_mesazone.shape[0]-df_mesazone.count())

## Task 3 Find zoning for each Denver County address

In [0]:
for i in range(len(df_mesazone)):
    if (df_mesazone['OppZoneGeoID'][i]=='00000000000'): df_mesazone['OppZoneGeoID'][i]=='0'
    if (df_mesazone['ZoningCode'][i]=='N/A'): df_mesazone['ZoningCode'][i]=='0'

## Task 4 Combine datasets

In [0]:
df_zone=pd.concat([df_mesazone,df_denzone])
print(df_zone.tail(),'\n')
print('Shape of the current dataframe:',df_zone.shape)

           City            FullAddress   Latitude   Longitude State  \
86069  COLLBRAN   2318 DIVIDE CREEK RD  39.356364 -107.585212    CO   
86070  COLLBRAN        75239 E HWY 330  39.359693 -107.678560    CO   
86071  COLLBRAN  24550 DIVIDE CREEK RD  39.361212 -107.588424    CO   
86072  COLLBRAN        75988 E HWY 330  39.364550 -107.643852    CO   
86073  COLLBRAN        75995 E HWY 330  39.365940 -107.646237    CO   

            StreetName StreetNumber    ZIP  
86069  DIVIDE CREEK RD         2318  816XX  
86070        E HWY 330        75239  816XX  
86071  DIVIDE CREEK RD        24550  816XX  
86072        E HWY 330        75988  816XX  
86073        E HWY 330        75995  816XX   

Shape of the current dataframe: (403926, 8)


In [0]:
## Save/Checkpoint
def write_checkpoint(df, dfname):
  filename=time.strftime("%Y%m%d-%H%M")+' '+ dfname+'.csv'
  df.to_csv(filename, index=False)
  return

def read_checkpoint(filename):
    return pd.read_csv(filename,index_col=0)
  
write_checkpoint(df_mesazone,'Mesa_Zone')
write_checkpoint(df_denzone,'Denver_Zone')
write_checkpoint(df_zone,'Both_Zone')

###### df_mesazone=read_checkpoint('20190331-0937 Mesa_Zone.csv')
###### df_mesazone.head()
